In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# API KEY : 코드 제출을 위해 환경변수 설정
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')
google_api_key = os.getenv('google_api_key')

In [2]:
import pandas as pd
df = pd.read_csv('기자간담회.csv')[['Date','Content']]
df = df.dropna()
df

,Date,Content
0,2025-01,공 보 관 - 지금부터 2025년 1월 통화정책방향에 대한 이창용 한국은행 총재의 ...
1,2025-02,공 보 관 - 지금부터 2025년 2월 통화정책방향에 대한 이창용 한국은행 총재의 ...
2,2025-04,공 보 관 - 지금부터 2025년 4월 통화정책방향에 대한 이창용 한국은행 총재의 ...
3,2025-05,공 보 관 - 지금부터 2025년 5월 통화정책방향에 대한 이창용 한국은행 총재의 ...
5,2025-07,공 보 관 - 지금부터 2025년 7월 통화정책방향에 대한 이창용 한국은행 총재의 ...


In [3]:
len(df['Content'][0].split(' '))

6574

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o'
                   , openai_api_key=api_key)  # 본인의 키로 변경

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

# 프롬프트 엔지니어링 : CO-STAR 프레임워크 사용
# 대형 언어 모델(LLM)의 프롬프트 엔지니어링을 체계화하기 위해 싱가포르 정부 기술청(GovTech Singapore)의 데이터 과학 및 인공지능 부서에서 개발한 구조화된 접근 방식
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """한국총재가 발표한 통화정책방향을 요약해주세요.
{context}
""")
])

# 체인 생성
chain = create_stuff_documents_chain(model, prompt)

# 요약 리스트
summaries = []

# Content 열 반복 처리
for content in df['Content']:
    doc = Document(page_content=content)
    result = chain.invoke({'context': [doc]})  # 문자열 반환됨
    summaries.append(result.strip())  # 바로 문자열로 저장

# 요약 추가
df['default_summary'] = summaries

In [15]:
print(df['default_summary'][0])  # 첫 번째 요약 확인

한국은행 이창용 총재는 기자간담회에서 금융통화위원회가 한국은행 기준금리를 3%로 유지하기로 결정했다고 발표했습니다. 그는 국내외 경제 여건과 그로 인한 불확실성을 고려하여 현재의 금리를 유지하고 더 많은 상황을 살펴보는 것이 적절하다고 판단했다고 설명했습니다. 금통위는 향후 경제상황에 따라 기준금리의 추가 인하 필요성을 고려하고 있으며, 특히 저신용 자영업자와 중소기업을 지원하기 위해 금융중개지원대출을 확대하기로 했습니다. 이와 관련하여 신성환 위원은 금리를 인하하는 것이 바람직하다는 소수 의견을 냈습니다.

총재는 대외 여건에서 미국 신정부의 정책 방향과 연준의 금리 인하 속도 등이 성장과 물가 전망의 불확실성을 증대시켰다고 설명했습니다. 또한, 국내 경기에서는 수출 증가세 둔화와 소비심리 위축으로 내수 회복세가 더딜 것으로 보이며, 지난 11월 성장률 전망치보다 하회할 가능성이 크다고 언급했습니다. 물가 상승률은 안정적 흐름을 보이고 있으나, 환율 상승이 상방 요인으로 작용할 것이라고 전했습니다.

결정의 배경으로 불확실성을 강조하며, 특히 정치적 리스크와 그로 인한 대외 신인도 문제를 지적하였습니다. 향후 통화정책 운용은 국내 대외의 경제정책 변화와 물가, 가계부채, 환율 등을 면밀히 점검하며 기준금리 추가 인하의 시기와 속도를 결정할 계획이라고 밝혔습니다.


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

# 프롬프트 엔지니어링 : CO-STAR 프레임워크 사용
# 대형 언어 모델(LLM)의 프롬프트 엔지니어링을 체계화하기 위해 싱가포르 정부 기술청(GovTech Singapore)의 데이터 과학 및 인공지능 부서에서 개발한 구조화된 접근 방식
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """당신은 글쓰기 전문가입니다. 아래 조건을 바탕으로 글을 작성해주세요:

1. Context:
- 주제는 한국은행 총재가 발표하는 통화정책방향입니다.

2. Objective:
- 독자가 장시간 발표된 내용을 요약본을 통해 편하게 인지할 수 있도록 합니다.

3. Style:
- 전문적인 글쓰기 스타일.

4. Tone:
- 격식 있고 신뢰감을 주는 톤.

5. Audience:
- 은행 직원으로 한국은행 총재의 발표 내용을 이해하고 요약할 수 있는 능력을 갖춘 사람들입니다.

6. Response:
- 가능한 한 심층 요약 글로 작성해주세요.
- 발표 내용을 주제별로 구분하여 항목별로 정리된 형태로 요약해주세요.
- 항목 제목은 발표 내용에 맞게 자유롭게 생성하되, 일관된 서식(숫자 또는 기호)을 사용해주세요.
- 발표에서 다룬 주제가 많을 경우 항목 수를 늘려도 좋습니다.

예시 형식:
1. ○○ 관련 발표 내용 요약  
2. △△ 배경 및 한국은행의 입장  
3. 향후 대응 방향

{context}
""")
])

# 체인 생성
chain = create_stuff_documents_chain(model, prompt)

# 요약 리스트
summaries = []

# Content 열 반복 처리
for content in df['Content']:
    doc = Document(page_content=content)
    result = chain.invoke({'context': [doc]})  # 문자열 반환됨
    summaries.append(result.strip())  # 바로 문자열로 저장

# 요약 추가
df['costar_summary'] = summaries

In [18]:
print(df['costar_summary'][0])

1. 금리 동결 배경 및 이유  
한국은행 금융통화위원회는 기준금리를 3%로 유지하기로 결정했습니다. 대내외 경제 여건의 불확실성 증가와 함께 미국 연준의 금리 인하 속도 조절 가능성을 고려하여, 현재 금리 수준을 유지하며 시장 상황을 면밀히 점검하기로 했습니다. 정치적 불확실성이 미치는 영향을 인식하여 성장의 하방 리스크에 대응하고자 신중한 접근이 필요하다는 판단이었습니다.

2. 대외 경제 여건과 전망  
세계 경제는 국가별로 혼재된 경기 흐름을 보이며, 미국의 견조한 성장세와 유로 지역의 둔화된 경기 회복세가 교차하고 있습니다. 중국은 부동산 경기 침체와 미 정부의 보호무역 정책 영향으로 인해 성장세가 둔화되고 있습니다. 또한, 유가 상승 등의 인플레이션 압력도 주요 변수로 작용하고 있습니다.

3. 국내 경제 상황  
12월 중 수출 증가율이 다소 높아졌지만, 내수 회복세가 기대에 미치지 못하고 있으며, 건설 투자 부진이 지속되고 있습니다. 경제 성장률은 2.2%와 1.9%의 전망치를 하회할 가능성이 큽니다. 정치적 불확실성과 환율 변동으로 인해 경제 불확실성이 증대되고 있습니다.

4. 물가와 금융시장 동향  
소비자물가는 석유류 가격 상승 등의 영향으로 다소 증가했지만, 근원물가 상승률은 소폭 하락했습니다. 앞으로도 환율 압력과 유가 움직임에 따라 물가가 높은 변동성을 보일 가능성이 큽니다. 원/달러 환율의 역풍과 금융시장의 변동성 확대가 두드러지며, 가계대출 둔화와 주택시장 조정도 지속되고 있습니다.

5. 향후 통화정책 운용 방향  
성장 리스크에 대응하여 추가 금리 인하가 검토될 예정입니다. 국내 정치 상황 변화, 주요국 경제정책의 전개 등을 지속적으로 점검하면서 금리 인하 시기와 속도를 결정할 것입니다. 이러한 맥락에서 내수 회복 지원을 위한 금융중개지원대출의 확대가 결정되었습니다.

6. 경제 전망 및 재정정책 제안  
경기 하강 국면에서 재정정책의 중요성이 강조되고 있습니다. 추경을 통한 일시적 경기 부양을 통해 자영업자 및 중소기업 지원

In [19]:
df.to_csv('기자간담회_요약.csv', index=False)  # 요약 결과를 CSV 파일로 저장

In [ ]:
import streamlit as st
import pandas as pd

# 페이지 제목
st.title("📅 기준금리 발표 요약 보기")

# 날짜 리스트로 버튼 생성
selected_date = st.radio("📌 확인하고 싶은 날짜를 선택하세요:", df['Date'].tolist())

# 선택된 날짜에 해당하는 요약 출력
summary = df.loc[df['Date'] == selected_date, 'costar_summary'].values[0]

# 결과 표시
st.markdown(f"### 📝 요약 내용 ({selected_date})")
st.write(summary)


100%|██████████| 5/5 [00:26<00:00,  5.21s/it]
